In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle

In [7]:
# Generar datos para el módulo de unidades
def generate_unit_data():
    x_data = []
    y_data = []
    for a in range(10):  # Primer número
        for b in range(10):  # Segundo número
            x_data.append([a, b])  # Entrada
            y_data.append((a + b) % 10)  # Unidades de la suma
    return np.array(x_data), np.array(y_data)

# Generar datos
x, y = generate_unit_data()

# Convertir etiquetas a one-hot
y_one_hot = to_categorical(y, num_classes=10)

# Dividir en conjuntos de entrenamiento y validación (80% entrenamiento)
train_size = int(len(x) * 0.8)  # 80% de datos para entrenamiento
#x_train, y_train = x[:train_size], y_one_hot[:train_size]
x_train, y_train = x, y_one_hot
x_val, y_val = x, y_one_hot   

## Asegurar formas consistentes para las entradas
x_train = np.expand_dims(x_train, axis=1)  # Agrega una dimensión para 'timesteps'
x_val = np.expand_dims(x_val, axis=1)      # Haz lo mismo con los datos de validación

# Construir el modelo
model = Sequential([
    LSTM(16, input_shape=(1, 2), return_sequences=True),
    LSTM(32, return_sequences=False),
    Dense(10, activation='softmax')  # Salida para 10 clases
])

# Callback personalizado para detener cuando todas las combinaciones sean correctas
class StopWhenPerfectCallback(EarlyStopping):
    def __init__(self, val_data, **kwargs):
        super().__init__(**kwargs)
        self.val_data = val_data  # Pasamos explícitamente los datos de validación
    
    def on_epoch_end(self, epoch, logs=None):
        # Evaluar el rendimiento sobre todo el conjunto de validación
        val_predictions = self.model.predict(self.val_data[0])
        val_pred_labels = np.argmax(val_predictions, axis=1)
        val_true_labels = np.argmax(self.val_data[1], axis=1)
        
        # Verificar si todas las combinaciones son correctas
        correct_predictions = np.sum(val_pred_labels == val_true_labels)
        total_predictions = len(val_true_labels)
        
        print(f'Evaluación de validación: {correct_predictions}/{total_predictions} correctas')

        # Si todas las combinaciones son correctas, detener el entrenamiento
        if correct_predictions == total_predictions:
            print("¡Todas las combinaciones han sido aprendidas correctamente! Deteniendo entrenamiento.")
            self.model.stop_training = True

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

# Crear una instancia del callback con los datos de validación
stop_callback = StopWhenPerfectCallback(val_data=(x_val, y_val), patience=500)

# Entrenar el modelo con el callback personalizado
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=500,
    batch_size=1,
    callbacks=[stop_callback]
)

# Evaluación final
loss, accuracy = model.evaluate(x_val, y_val)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Predicciones
predictions = model.predict(x_val)
predicted_numbers = np.argmax(predictions, axis=1)
real_numbers = np.argmax(y_val, axis=1)  # Etiquetas reales

print("Predicciones (probabilidades):", predictions[:5])
print("Predicciones (números):", predicted_numbers[:5])
print("Etiquetas reales:", real_numbers[:5])

# Mostrar las predicciones de validación y sus etiquetas
print("\nEvaluación de validación final:")
correct_predictions = np.sum(predicted_numbers == real_numbers)
total_predictions = len(real_numbers)
print(f'Predicciones correctas: {correct_predictions}/{total_predictions}')

# Guardar el modelo
model.save('unit_addition_module.keras')



Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step ep - accuracy: 0.0654 - loss: 2.3
Evaluación de validación: 10/100 correctas
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.0651 - loss: 2.3199 - val_accuracy: 0.1000 - val_loss: 2.3011
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step tep - accuracy: 0.1078 - loss: 2.306
Evaluación de validación: 11/100 correctas
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1029 - loss: 2.3069 - val_accuracy: 0.1100 - val_loss: 2.2977
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step tep - accuracy: 0.0724 - loss: 2.307
Evaluación de validación: 11/100 correctas
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0720 - loss: 2.3072 - val_accuracy: 0.1100 - val_loss: 2.2944
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step tep - accuracy: 0.1085 - loss: 2.298
Evaluación de validación: 15/100 correctas
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1076 - loss: 2.2988 - val_accuracy: 0.1500 - val_loss: 2.2795
Epoch 5/500
4/4 ━